In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from pprint import pprint

In [2]:
from platform import python_version
python_version()

'3.9.1'

### Aim

This notebook is created to explore and understand the events and items datasets, what columns they contain and what do they tell us. 

In [5]:
datapath = '../../datasets/user-items-recsys/'
category_tree = 'category_tree.csv'
events = 'events.csv'
item_props_1 = 'item_properties_part1.csv'
item_props_2 = 'item_properties_part2.csv'

In [6]:
df_events = pd.read_csv(datapath+events)
df_events.head()

,timestamp,visitorid,event,itemid,transactionid
0,1433221332117,257597,view,355908,NaN
1,1433224214164,992329,view,248676,NaN
2,1433221999827,111016,view,318965,NaN
3,1433221955914,483717,view,253185,NaN
4,1433221337106,951259,view,367447,NaN


In [7]:
df_category_tree = pd.read_csv(datapath+category_tree)
df_category_tree.head()

,categoryid,parentid
0,1016,213.0
1,809,169.0
2,570,9.0
3,1691,885.0
4,536,1691.0


In [8]:
df_items1 = pd.read_csv(datapath+item_props_1)
df_items1.head()

,timestamp,itemid,property,value
0,1435460400000,460429,categoryid,1338
1,1441508400000,206783,888,1116713 960601 n277.200
2,1439089200000,395014,400,n552.000 639502 n720.000 424566
3,1431226800000,59481,790,n15360.000
4,1431831600000,156781,917,828513


In [9]:
df_items2 = pd.read_csv(datapath+item_props_2)
df_items2.head()

,timestamp,itemid,property,value
0,1433041200000,183478,561,769062
1,1439694000000,132256,976,n26.400 1135780
2,1435460400000,420307,921,1149317 1257525
3,1431831600000,403324,917,1204143
4,1435460400000,230701,521,769062


### Dataset properties

*Events*
- Event type - View, AddToCart, Transaction
- Events table tells which event happened at what item on which item by which user

*Items*
- Tells about item category and value. An item can have its value changed over time. Hence, the table has timestamp column

*Category Tree*
- Denotes the heirarchy of category

### EDA - Events

In [10]:
df_events.shape

(2756101, 5)

In [11]:
df_events.columns

Index(['timestamp', 'visitorid', 'event', 'itemid', 'transactionid'], dtype='object')

In [12]:
## Convert timestamp to date
df_events['date'] = pd.to_datetime(df_events['timestamp'],unit='ms')

In [13]:
df_events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2756101 entries, 0 to 2756100
Data columns (total 6 columns):
 #   Column         Dtype         
---  ------         -----         
 0   timestamp      int64         
 1   visitorid      int64         
 2   event          object        
 3   itemid         int64         
 4   transactionid  float64       
 5   date           datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(3), object(1)
memory usage: 126.2+ MB


In [14]:
df_events['event'].value_counts()

view           2664312
addtocart        69332
transaction      22457
Name: event, dtype: int64

In [15]:
print("Total number of events: {}".format(df_events.shape[0]))
print("Number of unique users doing at least 1 event: {}".format(len(df_events['visitorid'].unique())))

Total number of events: 2756101
Number of unique users doing at least 1 event: 1407580


In [16]:
# Print minimum and maximum date
print("Minimum date in events table: ",df_events['date'].min())
print("Maximum date in events table: ",df_events['date'].max())

Minimum date in events table:  2015-05-03 03:00:04.384000
Maximum date in events table:  2015-09-18 02:59:47.788000


In [17]:
df_events.head()

,timestamp,visitorid,event,itemid,transactionid,date
0,1433221332117,257597,view,355908,NaN,2015-06-02 05:02:12.117
1,1433224214164,992329,view,248676,NaN,2015-06-02 05:50:14.164
2,1433221999827,111016,view,318965,NaN,2015-06-02 05:13:19.827
3,1433221955914,483717,view,253185,NaN,2015-06-02 05:12:35.914
4,1433221337106,951259,view,367447,NaN,2015-06-02 05:02:17.106


In [18]:
pprint("Each Transaction ID can have one of more items")
df_events['transactionid'].value_counts()

'Each Transaction ID can have one of more items'


7063.0     31
765.0      28
8351.0     27
2753.0     23
6993.0     21
           ..
16213.0     1
3504.0      1
5242.0      1
12477.0     1
5.0         1
Name: transactionid, Length: 10558, dtype: int64

### EDA - item properties

In [19]:
print("Items Data 1 ",df_items1.shape)
print("Items Data 2 ",df_items2.shape)

Items Data 1  (10999999, 4)
Items Data 2  (9275903, 4)


In [20]:
df_items1.columns

Index(['timestamp', 'itemid', 'property', 'value'], dtype='object')

In [39]:
## Convert timestamp to date
df_items1['datetime'] = pd.to_datetime(df_items1['timestamp'],unit='ms')
## Convert timestamp to date
df_items2['datetime'] = pd.to_datetime(df_items1['timestamp'],unit='ms')

In [22]:
df_items1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10999999 entries, 0 to 10999998
Data columns (total 5 columns):
 #   Column     Dtype         
---  ------     -----         
 0   timestamp  int64         
 1   itemid     int64         
 2   property   object        
 3   value      object        
 4   datetime   datetime64[ns]
dtypes: datetime64[ns](1), int64(2), object(2)
memory usage: 419.6+ MB


In [23]:
## Item properties can be - 'categoryid', 'price','name' etc. The property value is given in the values column
pprint("Below table shows that each item can have multiple properties")
pprint("It is possible that some of the properties change over time (e.g. price). Hence, we also have a timestamp column in the table")
df_items1[['itemid','property']].groupby(by=['itemid']).count()

'Below table shows that each item can have multiple properties'
('It is possible that some of the properties change over time (e.g. price). '
 'Hence, we also have a timestamp column in the table')


,property
itemid,
0,31
1,57
2,9
3,35
4,16
...,...
466862,10
466863,17
466864,42


**Let us pull one of the items and analyze its properties**

In [24]:
item = df_items1.query("itemid==1")
item.head()

,timestamp,itemid,property,value,datetime
208308,1433041200000,1,296,866110,2015-05-31 03:00:00
435957,1439694000000,1,59,769062,2015-08-16 03:00:00
977089,1433041200000,1,813,814966,2015-05-31 03:00:00
1765852,1433646000000,1,33,1128577 1000087 421694,2015-06-07 03:00:00
1804484,1433646000000,1,790,n5760.000,2015-06-07 03:00:00


In [25]:
item['property'].value_counts()

available     18
790           18
888            6
981            1
678            1
categoryid     1
689            1
776            1
0              1
296            1
59             1
284            1
839            1
764            1
185            1
33             1
813            1
364            1
Name: property, dtype: int64

The table above shows that property **790** has changed 18 times over time. 

In [27]:
# Analyzing the property "available" over time
item.query("property=='available'").sort_values(by="datetime")

,timestamp,itemid,property,value,datetime
6307952,1431226800000,1,available,1,2015-05-10 03:00:00
9687280,1431831600000,1,available,0,2015-05-17 03:00:00
6506736,1432436400000,1,available,0,2015-05-24 03:00:00
6705520,1433041200000,1,available,0,2015-05-31 03:00:00
6904304,1433646000000,1,available,0,2015-06-07 03:00:00
7103088,1434250800000,1,available,0,2015-06-14 03:00:00
7301872,1435460400000,1,available,0,2015-06-28 03:00:00
7898224,1436065200000,1,available,0,2015-07-05 03:00:00
8892144,1436670000000,1,available,0,2015-07-12 03:00:00
8295792,1437274800000,1,available,0,2015-07-19 03:00:00


The above table shows that itemid = 1 was available on 10 May but went out of stock after that.

### EDA - category tree

It denotes the heirarchy of categories. The categories are hashed

In [28]:
pprint(df_category_tree.shape)
df_category_tree.head()

(1669, 2)


,categoryid,parentid
0,1016,213.0
1,809,169.0
2,570,9.0
3,1691,885.0
4,536,1691.0


CategoryID **1016** is child of category **213**

In [29]:
df_category_tree.query("parentid==1532")

,categoryid,parentid
270,293,1532.0
273,986,1532.0
1066,107,1532.0
1539,500,1532.0
1603,1299,1532.0
1647,113,1532.0


In [30]:
df_category_tree.query("parentid==213")

,categoryid,parentid
0,1016,213.0
492,772,213.0
493,692,213.0
503,797,213.0
1088,1506,213.0
1089,1455,213.0


In [31]:
## Can one category have multiple parents??
maxSize = df_category_tree.groupby(by="categoryid").size().max()
print("Maximum size of a category group =",maxSize)

Maximum size of a category group = 1


Since maximum size of group is 1, each category has at most 1 parent

In [32]:
## How many categories does not have any parent (Root nodes)
df_root_nodes = df_category_tree[df_category_tree.parentid.isnull()]
print("There are total {} root nodes".format(df_root_nodes.shape[0]))
df_root_nodes.head()

There are total 25 root nodes


,categoryid,parentid
5,231,NaN
589,791,NaN
861,1490,NaN
939,431,NaN
1111,755,NaN


### Events - A little more

In [33]:
print("Total number of events:",df_events.shape[0])
print("Total number of transactions:",df_events.query("event=='transaction'").shape[0])
print("Total number of addToCart:",df_events.query("event=='addtocart'").shape[0])

Total number of events: 2756101
Total number of transactions: 22457
Total number of addToCart: 69332


In [34]:
## Get the users who have purchased at least one item
users_purchased = set(df_events.query("event=='transaction'").visitorid.unique())
print("Number of users who have purchased atleast once: ",len(users_purchased))

Number of users who have purchased atleast once:  11719


In [35]:
users_addtocart = set(df_events.query("event=='addtocart'").visitorid.unique())-users_purchased
print("Users who added to cart but never purchased: ",len(users_addtocart))

Users who added to cart but never purchased:  27146


In [36]:
users_viewonly = set(df_events.visitorid.unique()) - (users_addtocart.union(users_purchased))
print("Users who only viewed items:",len(users_viewonly))

Users who only viewed items: 1368715


### Item Properties - A little more

In [40]:
df_items = pd.concat([df_items1,df_items2])

In [42]:
n_items = df_items['itemid'].unique()
print("Number of unique items:",len(n_items))
print("Number of entries in items table:",df_items.shape[0])

Number of unique items: 417053
Number of entries in items table: 20275902


#### Analyzing item properties

#### Property value rules

1. String values are hashed<br>
2. Integers have 'n' preceding them

*To get property values which are integer, search for values starting with 'n'*

In [44]:
# Get Top 10 most frequent properties
df_items['property'].value_counts()[:10]

888           3000398
790           1790516
available     1503639
categoryid     788214
6              631471
283            597419
776            574220
678            481966
364            476486
202            448938
Name: property, dtype: int64

In [43]:
int_properties = df_items[df_items['value'].str.startswith('n')]
int_properties['property'].value_counts()[:5]

790    1790516
400     146749
888     128149
663      95594
917      91083
Name: property, dtype: int64

**Intuitions**:

**Property 888** could be the Name of the item as it is the most common.<br>
Also, it contains hashed values which is an indication of it being String.

**Property 790** has 100% values as integer. It is also the second most common property after 888.<br>
This is a good indication that this might be the price of the items

**Property availability** is a boolean property which denotes if the item is available at time t.<br>
This can be used in the final step as a filter in the recommendation based on item availability

**Property categoryid** indicates the item category.<br>
It can be used to look up in the heirarchy and find similar categories

In [46]:
item = 66480
properties = ['790','categoryid']
df_items.query("itemid == @item and property in @properties").sort_values(by="datetime")

,timestamp,itemid,property,value,datetime
7988211,1437274800000,66480,790,n71760.000,2015-05-10 03:00:00
8584560,1437879600000,66480,790,n71760.000,2015-05-24 03:00:00
8982126,1438484400000,66480,790,n71760.000,2015-05-24 03:00:00
7789428,1436670000000,66480,790,n75840.000,2015-05-31 03:00:00
8783343,1441508400000,66480,790,n75000.000,2015-05-31 03:00:00
6464885,1435460400000,66480,categoryid,1219,2015-06-28 03:00:00
9180909,1440903600000,66480,790,n75000.000,2015-06-28 03:00:00
700138,1436065200000,66480,790,n62040.000,2015-07-05 03:00:00
103789,1439089200000,66480,790,n71760.000,2015-08-09 03:00:00
8186994,1439694000000,66480,790,n71760.000,2015-08-09 03:00:00


The above table shows the price changes over time and categoryid of the item (itemid = 66480)<br>

#### Notes

Discount % - A good feature along with item price should be discount % wrt to the MRP<br>(We can assume maximum price at time t as MRP since we do not have that information)

### Questions?

1. *Can we check for similar users? In other words, can we create user embeddings?*

2. *Can we create item embedding?*

3. *Can we create an item-user embedding space?*

**Let us explore these questions in a separate notebook**